# XBeach Wave boundary

Demo notebook with usage examples

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import xarray as xr

from rompy_xbeach.boundary import (
    BoundaryStationSpectraJons,
)

import warnings
warnings.filterwarnings("ignore")

/home/rguedes/.virtualenvs/xbeach/lib/python3.10/site-packages/pyschism/forcing/hycom/gofs.py:8: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater as sw
/home/rguedes/.virtualenvs/xbeach/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "crs" in "SourceCRSWavespectra" shadows an attribute in parent "SourceMixin"
  warnings.warn(
/home/rguedes/.virtualenvs/xbeach/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "x_dim" in "SourceCRSWavespectra" shadows an attribute in parent "SourceMixin"
  warnings.warn(
/home/rguedes/.virtualenvs/xbeach/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "y_dim" in "SourceCRSWavespectra" shadows an attribute in parent "SourceMixin"
  warnings.warn(


In [3]:
datadir = Path("../../../rompy/tests/data")
outdir = Path("wave-boundary-demo")
outdir.mkdir(exist_ok=True, parents=True)

## Overview

XBeach wave boundary objects provide an interface to generate wave boundary conditions
from existing data.


### 1. Input data

#### 1.1. Input data structure

Two types of data structure are supported:

* **Gridded data**: The data are gridded over the (x, y) space. This type of data
usually includes integraged parameters but it is also possible to have gridded wave
spectra (e.g., ERA5 output).

* **Station data**: The data are defined with a single "station-type" dimension which
has associated variables to define the (x, y) coordinates of each station point.
Wavespectra are typically specified with this structure but wave parameters can also be
provided that way (e.g., SMC output).

#### 1.2. Input data type

Two types of data type are supported. These types may be specified from gridded and
station data structures.

* **Frequency-direction wave spectra**: Prescribe wave boundary conditions from the
full spectra

* **Integrated wave parameters**: Prescribe wave boundary conditions from integrated
wave parameters


### 2. XBeach `bctype`

XBeach can be run off wave spectra or wave parameters. The model uses the `bctype`
parameter to receive different types of wave boundary data.


### 3. Rompy-XBeach interface

The interface in rompy-xbeach allow constructing different `bctype` options from the
supported range of input data structures and data types. Each object is defined as
follows:

`Boundary{Data-structure}{Data-type}{Bctype}`

For example, the `BoundaryStationSpectraJons` interface deals with `station`, 2D
`spectra` data and produces `jons` bctype.

## Interfaces

XBeach boundary interfaces are intantiated with a `Source` instance plus some parameters
that are specific to each input data and bctype. They all have a `get()` method which
takes as inputs the workspace `destdir` directory, the model `grid`, and the model `time`.
The source data are sliced at the location of the offshore boundary, defined from the grid,
and over the model times. The sliced data is used to create XBeach `bcfile` data. The
method returns a dictionary with the key-value pair to write in the XBeach params file.

### BoundaryStationSpectraJons

XBeach jonswap type wave boundary from station spectra input data

In [4]:
from rompy.core.time import TimeRange
from rompy_xbeach.grid import RegularGrid
from rompy_xbeach.source import SourceCRSWavespectra
from rompy_xbeach.boundary import BoundaryStationSpectraJons

In [5]:
# Data source
source = SourceCRSWavespectra(uri=datadir / "aus-20230101.nc", reader="read_ww3")

# Model times
time = TimeRange(start="2023-01-01T00", end="2023-01-01T12", interval="1h")

# Model grid
grid = RegularGrid(
    ori=dict(x=115.594239, y=-32.641104, crs="epsg:4326"),
    alfa=347.0,
    dx=10,
    dy=15,
    nx=230,
    ny=220,
    crs="28350",
)

#### Multiple bcfiles


By default, multiple bcfiles with the associated filelist file are generated if the
multiple time steps are available in the source data within the time range provided.
This behaviour is controlled by the `filelist` parameter which defaults to True`.

In [6]:
destdir = outdir / "jons_multi"
destdir.mkdir(exist_ok=True)

# The filelist parameter parameter indicates one bcfile will be generated for each time
# step and a filelist file will be generated with the list of bcfiles. The coords
# parameter is usually not needed for wavespectra type sources since wavespectra datasets
# have established conventions for the coordinates. The dbtc parameter is specific to the
# Jons bctype and defines the internal XBeach timestep of the generated wave timeseries. 

wb = BoundaryStationSpectraJons(
    id="test",
    source=source,
    filelist=True, # this is the default value
    coords=dict(x="lon", y="lat", s="site"), # this is the default value
    dbtc=2.0, # timestep of the generated wave timeseries, ignore it to let xbeach decide
    fnyq=0.3, # Nyquist frequency, ignore it to let xbeach decide
    dfj=None, # step size frequency for the Jonswap spectrum, ignore it to let xbeach decide
)

# Generate the XBeach boundary files and return the list of bcfiles

namelist = wb.get(destdir=destdir, grid=grid, time=time)
namelist

{'filelist': 'filelist.txt'}

In [7]:
# The filelist.txt is created in the destdir directory and contains the list of bcfiles
# which are also created in the same workspace. One file is created for each time step
# in the input source data within the time period specified in the time parameter (the
# initial and end times are interpolated at time.start and time.end if these exact
# times are not present in the source dataset).

list(destdir.glob("*.txt"))

[PosixPath('wave-boundary-demo/jons_multi/filelist.txt'),
 PosixPath('wave-boundary-demo/jons_multi/jons-20230101T060000.txt'),
 PosixPath('wave-boundary-demo/jons_multi/jons-20230101T000000.txt')]

In [8]:
# Let's inspect the content of filelist.txt. Note the duration is defined by the time
# steps in the source dataset which in this test data file is 6h.

filelist = destdir / "filelist.txt"
print(filelist.read_text())

FILELIST
21600 2 jons-20230101T000000.txt
21600 2 jons-20230101T060000.txt



In [9]:
# And the content of one of the bcfiles

bcfile = destdir / filelist.read_text().split("\n")[1].split()[-1]
print(bcfile.read_text())

mainang = 44.6871
Hm0 = 0.451453
fnyq = 0.3
s = 66.9335
Tp = 14.3738
gammajsp = 2.37594



#### Single bcfile

If the `filelist` parameter is set to `False`, a single `bcfile` is created at `time.start`
and used to run the entire simulation period.

In [10]:
destdir = outdir / "jons_single"
destdir.mkdir(exist_ok=True)

# Generate a single bcfile at time.start by setting the filelist parameter to False.

wb = BoundaryStationSpectraJons(
    id="test",
    source=source,
    filelist=False
)

# Generate the XBeach boundary file, notice the 'bcfile' parameter is returned in the
# namelist in this case instead of 'filelist'.
#  
namelist = wb.get(destdir=destdir, grid=grid, time=time)
namelist

{'bcfile': 'jons-20230101T000000.txt'}

In [11]:
# The filelist file is not created in the workspace, only one single bcfile.

workspace_files = list(destdir.glob("*.txt"))
workspace_files

[PosixPath('wave-boundary-demo/jons_single/jons-20230101T000000.txt')]

In [12]:
print(workspace_files[0].read_text())

mainang = 44.6871
Hm0 = 0.451453
s = 66.9335
Tp = 14.3738
gammajsp = 2.37594

